# XL-BOMD
## see https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

In [1]:
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   280.01   1.447747e-01 -1.350262e+00 -1.205487e+00 ||    377.92   1.954009e-01 1.466852e+00 1.662253e+00 || 
     2   241.00   1.246049e-01 -1.330547e+00 -1.205942e+00 ||    338.37   1.749517e-01 1.487116e+00 1.662068e+00 || 
     3   192.14   9.934238e-02 -1.305328e+00 -1.205985e+00 ||    328.51   1.698518e-01 1.492053e+00 1.661905e+00 || 
     4   140.12   7.244645e-02 -1.277651e+00 -1.205204e+00 ||    347.02   1.794211e-01 1.482444e+00 1.661866e+00 || 
     5    91.10   4.710308e-02 -1.251810e+00 -1.204707e+00 ||    388.67   2.009560e-01 1.460626e+00 1.661582e+00 || 
     6    51.05   2.639654e-02 -1.230928e+00 -1.204532e+00 ||    445.53   2.303564e-01 1.430951e+00 1.661308e+00 || 
     7    25.35   1.310758e-02 -1.217638e+00 -1.204530e+00 ||    508.00   2.626565e-01 1.398344e+00 1.661000e+00 || 
     8    17.41   9.001947e-03 -1.213509e+00 -1.204507e+00 ||    566.01  

## Damping (Langevin)

In [4]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.
output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/4_XL-BOMD_damp'}
xl_bomd_params={'k':6}

md =  XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100.0,
              seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)
md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   285.79   1.477640e-01 -1.350202e+00 -1.202438e+00 ||    380.21   1.965849e-01 1.468534e+00 1.665119e+00 || 
     2   252.91   1.307640e-01 -1.329674e+00 -1.198910e+00 ||    364.07   1.882373e-01 1.489903e+00 1.678140e+00 || 
     3   212.76   1.100044e-01 -1.302967e+00 -1.192963e+00 ||    362.38   1.873650e-01 1.496224e+00 1.683589e+00 || 
     4   171.98   8.892239e-02 -1.271520e+00 -1.182598e+00 ||    394.36   2.038989e-01 1.487874e+00 1.691773e+00 || 
     5   123.99   6.410895e-02 -1.240546e+00 -1.176437e+00 ||    458.41   2.370160e-01 1.464583e+00 1.701599e+00 || 
     6    71.19   3.680733e-02 -1.213926e+00 -1.177119e+00 ||    515.47   2.665188e-01 1.436009e+00 1.702528e+00 || 
     7    29.85   1.543111e-02 -1.197917e+00 -1.182485e+00 ||    573.25   2.963936e-01 1.405441e+00 1.701834e+00 || 
     8    13.90   7.188450e-03 -1.192537e+00 -1.185349e+00 ||    611.36  